# Logistic Regression Example

1. Design model (input and output size, forward pass)
2. Construct loss and optimizer
3. Training loop:
    - Forward pass: compute predictio and loss
    - Backward pass: gradients
    - Update weights

# Code Implementation

In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
# Prepare dataset
bc = datasets.load_breast_cancer()

X, y = bc.data, bc.target

In [3]:
n_samples, n_features = X.shape

n_samples, n_features

(569, 30)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

In [5]:
# Scale features
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [6]:
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

In [7]:
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

In [8]:
# Model
class LogisticRegression(nn.Module):
    
    def __init__(self, num_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(num_input_features, 1)
        
    def forward(self, X):
        y_predicted = torch.sigmoid(self.linear(X))
        return y_predicted

model = LogisticRegression(n_features)

In [9]:
# Loss and optimizer
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [10]:
# Training loop
num_epochs = 100

for epoch in range(num_epochs):
    # forward pass and loss
    y_predicted = model(X_train)
    loss = criterion(y_predicted, y_train)
    
    # backward pass
    loss.backward()
    
    # update
    optimizer.step()
    
    optimizer.zero_grad()
    
    if (epoch + 1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss={loss.item(): .4f}')

epoch: 10, loss= 0.4644
epoch: 20, loss= 0.3876
epoch: 30, loss= 0.3383
epoch: 40, loss= 0.3039
epoch: 50, loss= 0.2785
epoch: 60, loss= 0.2588
epoch: 70, loss= 0.2430
epoch: 80, loss= 0.2300
epoch: 90, loss= 0.2191
epoch: 100, loss= 0.2098


In [11]:
with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    
    print(f'accuracy = {acc: .4f}')

accuracy =  0.9123
